In [81]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

In [82]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 1s 0us/step


In [83]:
# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

In [84]:
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [85]:
# Define the LeNet-5 model
model = Sequential([
    Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=(5, 5), activation='relu'),
    AveragePooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

In [86]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [87]:
# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 64s 45ms/step - loss: 0.3851 - accuracy: 0.8866 - val_loss: 0.1432 - val_accuracy: 0.9546
Epoch 2/10
469/469 [==============================] - 22s 47ms/step - loss: 0.1132 - accuracy: 0.9658 - val_loss: 0.0730 - val_accuracy: 0.9774
Epoch 3/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0811 - accuracy: 0.9750 - val_loss: 0.0701 - val_accuracy: 0.9754
Epoch 4/10
469/469 [==============================] - 21s 44ms/step - loss: 0.0641 - accuracy: 0.9803 - val_loss: 0.0501 - val_accuracy: 0.9848
Epoch 5/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0523 - accuracy: 0.9840 - val_loss: 0.0440 - val_accuracy: 0.9867
Epoch 6/10
469/469 [==============================] - 21s 44ms/step - loss: 0.0462 - accuracy: 0.9861 - val_loss: 0.0456 - val_accuracy: 0.9848
Epoch 7/10
469/469 [==============================] - 22s 47ms/step - loss: 0.0411 - accuracy: 0.9874 - val_loss: 0.0471 - val_accuracy:

In [88]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc}')

Test accuracy: 0.9890000224113464


In [89]:
# Save the trained model
model.save('mnist_model.h5')
print("Model saved successfully.")

Model saved successfully.


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [90]:
# Load the saved model
loaded_model = tf.keras.models.load_model('mnist_model.h5')
print("Model loaded successfully.")

Model loaded successfully.


In [91]:
# Example of using the loaded model for prediction
import numpy as np

In [92]:
# Assuming 'new_data' contains your new images
# Here, we'll use the first image from the test set as an example
new_data = np.expand_dims(x_test[0], axis=0)
predictions = loaded_model.predict(new_data)

1/1 [==============================] - 0s 144ms/step


In [93]:
# Print the predicted class probabilities
print("Predicted probabilities:", predictions)

Predicted probabilities: [[8.1026057e-09 1.4696965e-06 2.9030846e-07 8.2818742e-06 5.9947055e-08
  6.3338175e-09 1.3768115e-13 9.9997950e-01 2.8903420e-08 1.0502149e-05]]


In [94]:
# Print the predicted class
predicted_class = np.argmax(predictions)
print("Predicted class:", predicted_class)

Predicted class: 7


In [95]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [110]:
# Define constants
img_height, img_width = 224, 224
batch_size = 32
num_classes = 2  # Update with the number of classes in your dataset
dataset_path = "/content/drive/MyDrive/Dataset/Pistachio_Image_Dataset"

In [98]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [99]:
# Load and prepare the training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # Set as training data

Found 1719 images belonging to 3 classes.


In [101]:
# Load and prepare the validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # Set as validation data

Found 429 images belonging to 3 classes.


In [102]:
# Load pre-trained VGG19 model
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [103]:
# Freeze convolutional layers
for layer in vgg19_model.layers:
    layer.trainable = False

In [104]:
# Add custom classification layers
model = tf.keras.models.Sequential([
    vgg19_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [105]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [106]:
# Print the model summary
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_17 (Flatten)        (None, 25088)             0         
                                                                 
 dense_51 (Dense)            (None, 256)               6422784   
                                                                 
 dropout_22 (Dropout)        (None, 256)               0         
                                                                 
 dense_52 (Dense)            (None, 2)                 514       
                                                                 
Total params: 26447682 (100.89 MB)
Trainable params: 6423298 (24.50 MB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


In [111]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la

In [ ]:
# Save the trained model
model.save('vgg19_pistachio_model.h5')